## <span style="color:blue"> Our Imports</span>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import serial
import time


## <span style="color:blue"> Importing Our dataset</span>

In [ ]:
dfs = pd.read_csv(r'C:/Users/ashra/OneDrive/Documents/instrumentation projects/project2/Dataset/user_b.csv')

In [ ]:
dfs
dfs.shape[0]
AmplitudeValues = dfs.iloc[:, 6]

In [ ]:
from PyQt5 import QtWidgets, uic
import sys
class MainApp(QtWidgets.QMainWindow):
    def __init__(self):
        super(MainApp, self).__init__()
        uic.loadUi('main.ui', self)  # Load the UI file
        self.time = np.linspace(0, 1, dfs.shape[0])  # Create 1000 evenly spaced time points
        self.plotEEG()
    def plotEEG(self):
        self.widget_signal.plot(self.time, AmplitudeValues)

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = MainApp()
    window.show()
    sys.exit(app.exec_())


## <span style="color:blue"> Shuffling Data</span>

In [ ]:
data = dfs.sample(frac=1.0, random_state=123).reset_index(drop=True)

In [ ]:
data

## <span style="color:blue"> Our Model</span>

In [ ]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()

    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

    return X_train, X_test, y_train, y_test, scaler  # Return scaler here

In [ ]:
def build_model(num_classes=3):

    inputs = tf.keras.Input(shape=(X_train.shape[1],))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
X_train, X_test, y_train, y_test, scaler = preprocess_inputs(data, target='Class')  # Get scaler here

In [ ]:
class_model = build_model(num_classes=3)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

## <span style="color:blue">Accuracy</span>

In [ ]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

In [ ]:
# Load new data
new_data = X_test

In [ ]:
new_predictions = class_model.predict(new_data)

predicted_classes = np.argmax(new_predictions, axis=1)

predicted_classes_list = predicted_classes.tolist()


In [ ]:
predicted_classes

In [ ]:
predicted_classes_list

In [ ]:
list(y_test)
len(predicted_classes_list)

## <span style="color:blue">Link With Arduino</span>

In [ ]:
# !conda install -c anaconda pyserial -y

In [ ]:
# Define the serial port and baud rate (adjust as needed)
ser = serial.Serial('COM5', 9600)  # Change 'COM5' to the appropriate port name on your system

# Your list containing values 0, 1, or 2
# motor_commands = predicted_classes_list[:21]
# motor_commands = predicted_classes_list
motor_commands = [0,0,0,0,1,1,1,2,2,2]

try:
    for command in motor_commands:
        # Send the command to the Arduino
        ser.write(str(command).encode())
        time.sleep(2)  # Adjust the delay as needed
except KeyboardInterrupt:
    pass

ser.close()

In [ ]:
predicted_classes_list[:21]